<a href="https://colab.research.google.com/github/ramayer/google-colab-examples/blob/main/Apache_Spark_with_Delta_Tables_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apache Spark with Delta Tables in Google Colab


#### Install Java

In [ ]:
!apt-get install -y -qq openjdk-8-jdk-headless 

#### install Spark

In [ ]:
!(wget -q --show-progress -nc https://mirrors.ocf.berkeley.edu/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz)
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

## Install pyspark and related python libraries



In [ ]:
try:
  import pyspark, findspark, delta
except:
  %pip install -q --upgrade pyspark
  %pip install -q findspark
  %pip install -q delta

# Start a Spark Session


In [ ]:
import findspark
import pyspark
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

MAX_MEMORY="8g"
findspark.init()
from pyspark.sql import SparkSession
spark = (pyspark.sql.SparkSession.builder.appName("MyApp") 
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") 
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
    .config("spark.executor.memory", MAX_MEMORY) 
    .config("spark.driver.memory", MAX_MEMORY) 
    .enableHiveSupport() 
    .getOrCreate()        
    )

spark

## Create a delta table

In [ ]:
#from delta.tables import DeltaTable
import delta

df = spark.createDataFrame([{'hi':'hello','w':'world'}])

(df.write.format('delta')
         .mode('overwrite')
         .option("mergeSchema", "true")
         .save('./delta_hello_world')
)


## Query the delta table

In [42]:
spark.read.format("delta").load('./delta_hello_world').createOrReplaceTempView("delta_hello_world")
df2 = spark.sql("""
  select * from delta_hello_world
""")
df2.toPandas()

,hi,w
0,hello,world


# Save dataframe on google drive

In [31]:
from google.colab import drive
if not os.path.ismount('/content/gdrive'):
  drive.mount('/content/gdrive')
else:
  print("drive was already mounted")

Mounted at /content/gdrive


In [40]:
df.write.format("delta").save("/content/gdrive/MyDrive/Colab Datasets/test_delta_table")